In [3]:
# This is a practice Artificial Neural Network
# The problem being solved is based off of a model bank with fake data
# The bank has customers that have left for whatever reason
# The goal is to find why these customers have left using information such as account balance and gender
# The last column of the data states whether or not the customer has left the bank

import os # Navigating directories
import nibabel as nib # For nii files

import numpy as np #Math operations library
import matplotlib.pyplot as plt #Visualization library
import pandas as pd #Matrix handler

import keras # Brings in tensorflow with it
from keras.models import Sequential # Used for initialization of ANN
from keras.layers import Dense, Conv3D, MaxPooling3D, Flatten # adds layers to ANN
from keras.wrappers.scikit_learn import KerasClassifier # ability to turn network into a function definition
from keras.layers import Dropout

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler #Methods to change categorical strings to numbers and scaling ability
from sklearn.model_selection import train_test_split # Splits data into training and testing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix # Creates truth table for evaluating results

def build_classifier(nodes, hidden_layers) :
    classifier = Sequential() # This is the ANN object
    classifier.add(Conv3D(32, (3, 3, 3), input_shape=(79, 95, 79, 1), strides=(1, 1, 1), padding='valid', data_format=None, dilation_rate=(1, 1, 1), activation='relu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
                   #Conv3D(32, (3, 3, 3), input_shape=(79, 95, 79), activation='relu')) # 32 kernels, 3x3; input image is rgb and 64x64
    #Note: images are not the same size and will need adjustment prior to training
    #Note: docs using Theano backend reference the input shape with the channel number first
    classifier.add(MaxPooling3D(pool_size=(2, 2, 2))) # Mix a 4x4 square into a 1x1
    classifier.add(Conv3D(32, (3, 3, 3), activation='relu'))
    classifier.add(MaxPooling3D(pool_size=(2, 2, 2)))
    classifier.add(Flatten())
    
    classifier.add(Dense(input_dim=11, units=nodes, kernel_initializer='uniform', activation='relu')) #Creates first hidden layer
    classifier.add(Dropout(rate=0.1))
    
    for i in range(0, hidden_layers):
        classifier.add(Dense(units=nodes, kernel_initializer='uniform', activation='relu')) # Second layer. Input dim is known from previous layer
        classifier.add(Dropout(rate=0.1))
    
    classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid')) # Output layer. Only 1 ouput category, sigmoid activation to get probability of sureness
    classifier.add(Dropout(rate=0.1))

    # Note: Softmax applies to a dependent variable that has more than 2 categories
    # i.e. fMRI categorizations

    classifier.compile(optimizer='rmsprop', loss="binary_crossentropy", metrics=['accuracy'])
    # Notes
    # adam is a kind of stochastic gradient descent
    # For multivariabel, use categorical cross entropy
    # Accuracy is predefined
    return classifier
# Creates a standard Keras type classifier composed of the defined network for
# k-means testing
parameters = {'batch_size': 25,
              'epochs': 500,
              'optimizer': 'rmsprop', #'adam'],
              'nodes': 6,
              'hidden_layers': 1} # Creates list of parameters to test to find most successful one


classifier = build_classifier(parameters['nodes'], parameters['hidden_layers'])

In [ ]:
# model.train_on_batch(x_batch, y_batch)
# loss_and_metrics = model.evaluate(x_test, y_test, batch_size=128)
# classes = model.predict(x_test, batch_size=128)

rootDir = '/media/jasondent/SANDISK/PNC_resting_state'
counter = 0
for dirName, subdirList, fileList in os.walk(rootDir):
    print('Found directory: %s' % dirName)
    for fname in fileList:
        file_location = dirName + '/' + fname
        img = nib.load(file_location)
        data = img.get_fdata()
        # print(np.max(img.get_fdata()))
        counter = counter + 1
        # print(img.get_fdata().shape())
        data.resize(1, 79, 95, 79, 1)
        print(data.shape)
        classifier.train_on_batch(data, np.array([0]))
        break


Found directory: /media/jasondent/SANDISK/PNC_resting_state
Found directory: /media/jasondent/SANDISK/PNC_resting_state/600710344290
(1, 79, 95, 79, 1)
Found directory: /media/jasondent/SANDISK/PNC_resting_state/600730262874
(1, 79, 95, 79, 1)
Found directory: /media/jasondent/SANDISK/PNC_resting_state/600745453476
(1, 79, 95, 79, 1)
